In [1]:
import torch.cuda

from src import  N_CLASSES, FEATURE_DIM

BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cpu


In [2]:
# 读数据
data = torch.load("data.pth")
feature = data['feature']
label = data['label']

In [3]:
# 生成训练集、测试集
from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, train_size=0.8, random_state=0)
# x_train, x_test, y_train, y_test = x_train.to(device), x_test.to(device), y_train.to(device), y_test.to(device)
x_train = feature.to(device)
y_train = label.to(device)

In [4]:
# 加载数据
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# test_dataset = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 定义网络
from model import Net
import torch.nn as nn

net = Net().to(device)

# 定义损失函数、优化器
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),
                       lr=0.0005,
                       betas=(0.9, 0.999),
                       eps=1e-08,
                       weight_decay=0,
                       amsgrad=False)

In [6]:
# 训练网络
for epoch in range(600):  # 数字越大，循环次数越多
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    if epoch % 10 == 0:
        print(f'epoch {epoch} loss: {running_loss / 1500:.10f}')
        running_loss = 0.0

print('Finished Training')

# 训练结束后保存网络参数
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

epoch 0 loss: 0.4780592679
epoch 10 loss: 0.0776317313
epoch 20 loss: 0.0500846855
epoch 30 loss: 0.0394945701
epoch 40 loss: 0.0288851603
epoch 50 loss: 0.0276010668
epoch 60 loss: 0.0229669614
epoch 70 loss: 0.0186456444
epoch 80 loss: 0.0184432390
epoch 90 loss: 0.0155899879
epoch 100 loss: 0.0156943521
epoch 110 loss: 0.0143125583
epoch 120 loss: 0.0122162846
epoch 130 loss: 0.0120155831
epoch 140 loss: 0.0106495676
epoch 150 loss: 0.0123110038
epoch 160 loss: 0.0119060605
epoch 170 loss: 0.0092616656
epoch 180 loss: 0.0095954233
epoch 190 loss: 0.0099982194
epoch 200 loss: 0.0085050230
epoch 210 loss: 0.0084994676
epoch 220 loss: 0.0084203183
epoch 230 loss: 0.0092053216
epoch 240 loss: 0.0079403041
epoch 250 loss: 0.0061410896
epoch 260 loss: 0.0076607621
epoch 270 loss: 0.0069279098
epoch 280 loss: 0.0068134812
epoch 290 loss: 0.0063051897
epoch 300 loss: 0.0082232788
epoch 310 loss: 0.0058198387
epoch 320 loss: 0.0054563015
epoch 330 loss: 0.0056804747
epoch 340 loss: 0.0067682

In [ ]:
# 测试模型
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # noinspection PyShadowingBuiltins
    for input, labels in test_loader:
        # calculate outputs by running images through the network
        outputs = net(input)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # noinspection PyUnresolvedReferences
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total}')